In [1]:
import sys, os, inspect, logging, importlib

import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx

import matplotlib.pyplot as plt

sys.path.append(r'../../GOSTNets/GOSTNets')
import GOSTnet as gn

from shapely.geometry import Point, LineString

In [2]:
# try reading graph from shp file


In [3]:
scenario_var = "_Feb_v5"
cost_attr = 'cost'
res_folder = r"./"
G_pickle  = os.path.join(res_folder, "asia_railways_ver3.pickle")

# Read in and process input data
edges_geography = r"./geographic_data/edges.shp"
edges_data = gpd.read_file(edges_geography)

# Output files
complete_volume_map = os.path.join(res_folder, "complete_equal_volume_map_Feb_v5.csv")
trade_volume_map = os.path.join(res_folder, "trade_volume_map_Feb_v5.csv")

# need this for now...but instead show just read in nodes from edges?
nodes_geography = r"./geographic_data/rail_nodes_Feb_v5.csv"
nodes_data = pd.read_csv(nodes_geography, dtype = {'X': np.float64, 'Y': np.float64})
nodes_data['Point'] = [Point(x, y) for x, y in zip(nodes_data['X'],nodes_data['Y'])]
nodes_data['node_id'] = nodes_data['node_id'].astype(str)

OD_df = pd.read_csv(os.path.join(res_folder, "asia_railways_output_OD_matrix{}.csv".format(scenario_var)), index_col=0)
G = nx.read_gpickle(G_pickle)

trade_flows = pd.read_csv(os.path.join(res_folder, "trade_flows_Feb_v5.csv"))

In [4]:
len(trade_flows)

100

In [5]:
edges_data

,source,target,link_id,length,time,name_x,geometry
0,38,69,1,1304,28.977778,Kashgar,"LINESTRING (75.99371 39.46769, 89.18467 42.94998)"
1,40,69,29,791,17.577778,Khorgos,"LINESTRING (80.41190 44.21607, 89.18467 42.94998)"
2,38,70,2,174,3.866667,Kashgar,"LINESTRING (75.99371 39.46769, 75.41076 40.57480)"
3,36,70,3,259,10.360000,Kara Suu,"LINESTRING (72.86264 40.70247, 75.41076 40.57480)"
4,36,49,104,22,0.880000,Kara Suu,"LINESTRING (72.86264 40.70247, 72.81063 40.52017)"
...,...,...,...,...,...,...,...
104,20,91,114,1560,47.640000,None,"LINESTRING (60.64593 25.29688, 72.92575 19.02477)"
105,12,66,19,623,26.520000,None,"LINESTRING (52.23207 30.26452, 59.21328 35.27869)"
106,45,63,88,1040,44.770000,None,"LINESTRING (59.61583 36.26611, 51.42322 35.69698)"
107,20,92,115,3000,1091.620000,None,"LINESTRING (60.64593 25.29688, 60.76567 14.76131)"


In [6]:
nodes_data[:18]

,node_id,name,country,border,gauge_change,if_border_is_an_origin_or_destination,Name,X,Y,Point
0,1,Aktau,KAZ,False,False,False,Aktau,51.194236,43.650086,POINT (51.19423615 43.6500856)
1,2,Aktobe,KAZ,False,False,False,Aktobe,57.166930,50.283883,POINT (57.16692971 50.28388335)
2,3,Aktogal,KAZ,False,False,False,Aktogal,79.922600,46.851951,POINT (79.92260014999999 46.8519507)
3,4,Almaty,KAZ,False,False,False,Almaty,76.857735,43.229664,POINT (76.85773523 43.22966422)
4,5,Andijan,UBZ,True,False,False,Andijan,72.284590,40.814750,POINT (72.28458996000001 40.8147501)
5,6,Angren,UBZ,False,False,False,Angren,70.074230,41.008453,POINT (70.07422952 41.00845288)
6,7,Arys,KAZ,False,False,False,Arys,68.813051,42.433659,POINT (68.8130505 42.43365853)
7,8,Ashgabat,TKM,False,False,False,Ashgabat,58.316298,37.943921,POINT (58.31629783 37.94392081)
8,9,Atrek,TKM,False,False,False,Atrek,54.021612,37.414975,POINT (54.02161162 37.41497489)
9,10,Baku,AZE,True,False,True,Baku,49.866373,40.409438,POINT (49.86637319 40.40943762)


In [7]:
OD_df

,38,69,70,36,5,25,35,54,37,65,...,80,81,89,93,94,90,91,92,18b,68b
38,0.171170,42.271170,17.641170,69.156170,107.971170,122.511170,137.791170,141.701170,148.141170,160.201170,...,334.591170,399.401170,249.111170,304.451170,334.331170,429.221170,475.391171,541.021170,340.711170,183.231171
69,42.271170,0.171170,59.741170,111.256170,130.181170,115.641170,118.921170,115.011170,121.451170,133.511170,...,307.901170,372.711170,222.421170,277.761170,307.641170,402.531170,448.701171,514.331170,298.611170,156.541171
70,17.641170,59.741170,0.171170,51.686170,90.501170,105.041170,120.321170,124.231170,130.671170,142.731170,...,317.121170,381.931170,231.641170,286.981170,316.861170,411.751170,457.921171,523.551170,358.181170,165.761171
36,69.156170,111.256170,51.686170,0.171170,38.986170,53.526170,68.806170,72.716170,79.156170,91.216170,...,265.606170,330.416170,180.126170,235.466170,265.346170,360.236170,406.406171,472.036170,322.656170,114.246171
5,107.971170,130.181170,90.501170,38.986170,0.171170,14.711170,29.991170,33.901170,40.341170,52.401170,...,226.791170,291.601170,141.311170,196.651170,226.531170,321.421170,367.591171,433.221170,283.841170,75.431171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,429.221170,402.531170,411.751170,360.236170,321.421170,306.881170,291.601170,287.691170,281.251170,269.191170,...,125.821170,61.011170,180.281170,124.941170,95.061170,0.171170,471.801170,202.631170,556.191170,319.881171
91,475.391171,448.701171,457.921171,406.406171,367.591171,353.051171,337.771171,333.861171,330.961171,343.021171,...,346.151170,410.961170,365.281170,420.621170,450.501170,471.801170,0.171170,552.581170,574.071171,292.331170
92,541.021170,514.331170,523.551170,472.036170,433.221170,418.681170,403.401170,399.491170,393.051170,380.991170,...,206.601170,141.791170,351.961170,327.401170,297.521170,202.631170,552.581170,0.171170,667.991170,431.681171
18b,340.711170,298.611170,358.181170,322.656170,283.841170,269.301170,272.581170,268.671170,275.111170,287.171170,...,461.561170,526.371170,376.081170,431.421170,461.301170,556.191170,574.071171,667.991170,0.171170,310.201171


# Map network volume based on ALL least-cost trips

In [8]:
def tabulate_edges(route):
    edge_table = []
    for idx in range(0, len(route) - 1):
        edge_table.append([route[idx], route[idx+1]])
    return(edge_table)

"""
def get_route(route, nodes_data):
    all_nodes = []
    for x in route:
        try:
            cur_node = nodes_data.loc[nodes_data['node_id'] == x]
            all_nodes.append(Point(cur_node.X, cur_node.Y))
        except:
            pass
    if len(all_nodes) > 1:
        return(LineString(all_nodes))
    else:
        return(None)
"""

"\ndef get_route(route, nodes_data):\n    all_nodes = []\n    for x in route:\n        try:\n            cur_node = nodes_data.loc[nodes_data['node_id'] == x]\n            all_nodes.append(Point(cur_node.X, cur_node.Y))\n        except:\n            pass\n    if len(all_nodes) > 1:\n        return(LineString(all_nodes))\n    else:\n        return(None)\n"

In [9]:
all_res = []
complete_edges = []

count = 0

for origin, row in OD_df.iterrows():    
    for destination, value in row.items():
        try:
            origin = int(origin)
        except:
            pass
        try:
            destination = int(destination)
        except:
            pass
        
        count = count + 1
        route = nx.bidirectional_shortest_path(G, origin, destination)
        # print(route)
        edge_table = tabulate_edges(route)
        print(edge_table)
        complete_edges = complete_edges + edge_table 
        #all_res.append([origin, destination, get_route(route, nodes_data)])


[]
[[38, 69]]
[[38, 70]]
[[38, 70], [70, 36]]
[[38, 70], [70, 36], [36, 5]]
[[38, 70], [70, 36], [36, 5], [5, 25]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 37]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 37], [37, 65]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 37], [37, 65], [65, 31]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39], [39, 68]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39], [39, 68], [68, '68b'], ['68b', 43]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58], [58, 55]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35]

[[43, 8], [8, 67], [67, 10], [10, 1], [1, 71]]
[[43, 8], [8, 67], [67, 10], [10, 1], [1, 71], [71, 86]]
[[43, 8], [8, 67], [67, 10], [10, 1], [1, 71], [71, 86], [86, 30]]
[[43, 8], [8, 67], [67, 10], [10, 1], [1, 71], [71, 86], [86, 30], [30, 14]]
[[43, 58], [58, 55], [55, 45], [45, 26], [26, 57], [57, 28], [28, 85], [85, 9]]
[[43, 58], [58, 55], [55, 45], [45, 26], [26, 57], [57, 28], [28, 85]]
[[43, 58], [58, 55], [55, 45], [45, 26], [26, 57], [57, 28]]
[[43, 58], [58, 55], [55, 45], [45, 26], [26, 57]]
[[43, 58], [58, 55], [55, 45], [45, 26]]
[[43, 8], [8, 67], [67, 10]]
[[43, 8], [8, 67], [67, 10], [10, 62]]
[[43, 8], [8, 67], [67, 10], [10, 62], [62, 84]]
[[43, 8], [8, 67], [67, 10], [10, 62], [62, 82]]
[[43, 8], [8, 67], [67, 10], [10, 62], [62, 82], [82, 34]]
[[43, 8], [8, 67], [67, 10], [10, 62], [62, 82], [82, 34], [34, 32]]
[[43, 8], [8, 67], [67, 10], [10, 62], [62, 82], [82, 34], [34, 73], [73, 18]]
[[43, 8], [8, 67], [67, 10], [10, 62], [62, 82], [82, 34], [34, 73], [73, 1

[[42, 59], [59, 33], [33, 83], [83, 66], [66, 45], [45, 26], [26, 57], [57, 28]]
[[42, 59], [59, 33], [33, 83], [83, 66], [66, 45], [45, 26], [26, 57]]
[[42, 59], [59, 33], [33, 83], [83, 66], [66, 45], [45, 26]]
[[42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 10]]
[[42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 10], [10, 62]]
[[42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 10], [10, 62], [62, 84]]
[[42, 59], [59, 33], [33, 83], [83, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 82]]
[[42, 59], [59, 33], [33, 83], [83, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34]]
[[42, 59], [59, 33], [33, 83], [83, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 32]]
[[42, 59], [59, 33], [33, 83], [83, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 73], [73, 18]]
[[42, 31], [31, 65], [

[[7, 53], [53, 2], [2, 87], [87, 48]]
[[7, 53], [53, 2], [2, 87]]
[[7, 53], [53, 2]]
[[7, 53], [53, 16], [16, 1], [1, 10], [10, 62], [62, 82], [82, 34], [34, 73], [73, 18], [18, 75]]
[[7, 53], [53, 16], [16, 1], [1, 10], [10, 62], [62, 82], [82, 34], [34, 32], [32, 15], [15, 88]]
[[7, 53], [53, 16], [16, 1], [1, 10], [10, 62], [62, 82], [82, 34], [34, 32], [32, 15]]
[[7, 17], [17, 4], [4, 21]]
[[7, 17], [17, 4], [4, 21], [21, 3]]
[[7, 17], [17, 4], [4, 21], [21, 3], [3, 11]]
[[7, 53], [53, 78], [78, 77], [77, 46]]
[[7, 53], [53, 78], [78, 77]]
[[7, 53], [53, 78]]
[[7, 53], [53, 16], [16, 1], [1, 10], [10, 62], [62, 82], [82, 34], [34, 61], [61, 72], [72, 52]]
[[7, 53], [53, 16], [16, 1], [1, 10], [10, 62], [62, 82], [82, 34], [34, 61], [61, 72]]
[[7, 53], [53, 16], [16, 1], [1, 10], [10, 62], [62, 82], [82, 34], [34, 61]]
[[7, 53], [53, 16], [16, 1], [1, 10], [10, 67], [67, 8]]
[[7, 53], [53, 16], [16, 1], [1, 10], [10, 67]]
[[7, 60]]
[[7, 60], [60, 54], [54, 39], [39, 68], [68, '68b']

[[30, 86], [86, 71], [71, 1], [1, 16], [16, 53], [53, 7], [7, 60], [60, 6]]
[[30, 86], [86, 71], [71, 1], [1, 16], [16, 53], [53, 7], [7, 60], [60, 6], [6, 50]]
[[30, 86], [86, 71], [71, 1], [1, 10], [10, 62], [62, 51]]
[[30, 86], [86, 71], [71, 1], [1, 10], [10, 62], [62, 51], [51, 47]]
[[30, 86], [86, 71], [71, 1], [1, 10], [10, 62], [62, 51], [51, 47], [47, 41]]
[[30, 86], [86, 71], [71, 1], [1, 10], [10, 62], [62, 82], [82, 34], [34, 73]]
[[30, 86], [86, 71], [71, 1], [1, 16], [16, 53], [53, 7], [7, 60], [60, 54], [54, 37], [37, 22]]
[[30, 86], [86, 71], [71, 1], [1, 16], [16, 53], [53, 7], [7, 60], [60, 54], [54, 35], [35, 25], [25, 5], [5, 36], [36, 49]]
[[30, 14], [14, 9], [9, 85], [85, 28], [28, 57], [57, 26], [26, 45], [45, 66], [66, 83], [83, 33], [33, 79]]
[[30, 14], [14, 9], [9, 85], [85, 28], [28, 57], [57, 26], [26, 45], [45, 66], [66, 83], [83, 33], [33, 79], [79, 80]]
[[30, 14], [14, 9], [9, 85], [85, 28], [28, 57], [57, 26], [26, 45], [45, 66], [66, 83], [83, 33], [33,

...

[[22, 37], [37, 54], [54, 35], [35, 25], [25, 5], [5, 36], [36, 70]]
[[22, 37], [37, 54], [54, 35], [35, 25], [25, 5], [5, 36]]
[[22, 37], [37, 54], [54, 35], [35, 25], [25, 5]]
[[22, 37], [37, 54], [54, 35], [35, 25]]
[[22, 37], [37, 54], [54, 35]]
[[22, 37], [37, 54]]
[[22, 37]]
[[22, 37], [37, 65]]
[[22, 37], [37, 65], [65, 31]]
[[22, 37], [37, 39]]
[[22, 37], [37, 39], [39, 68]]
[[22, 37], [37, 39], [39, 68], [68, '68b'], ['68b', 43]]
[[22, 37], [37, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58]]
[[22, 37], [37, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58], [58, 55]]
[[22, 37], [37, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58], [58, 55], [55, 45]]
[[22, 37], [37, 65], [65, 31], [31, 42], [42, 59], [59, 33], [33, 83], [83, 66]]
[[22, 37], [37, 65], [65, 31], [31, 42], [42, 59], [59, 33], [33, 83], [83, 66], [66, 12]]
[[22, 37], [37, 65], [65, 31], [31, 42], [42, 59], [59, 33], [33, 83], [83, 66], [66, 12], [12, 76]]
[[22, 37], [37, 65], [65, 31], [31, 42], [42, 59], [5

[[79, 33], [33, 83], [83, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, 8], [8, 67]]
[[79, 33], [33, 59], [59, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60]]
[[79, 33], [33, 83], [83, 66], [66, 45], [45, 63]]
[[79, 33], [33, 59], [59, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 6]]
[[79, 33], [33, 59], [59, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 6], [6, 50]]
[[79, 33], [33, 83], [83, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, 8], [8, 67], [67, 10], [10, 62], [62, 51]]
[[79, 33], [33, 83], [83, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 32], [32, 41], [41, 47]]
[[79, 33], [33, 83], [83, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 32], [32, 41]]
[[79, 33], [33, 83], [83, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 73]]
[[79, 33], [33, 59], [59, 42], [42, 31], [31, 65], [65, 37], [37, 22]]
[[79, 33], [33, 59], [59, 42], [42, 31], [31, 65], [65, 37], [37, 5

[[89, 42], [42, 59], [59, 33], [33, 83], [83, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 82]]
[[89, 42], [42, 59], [59, 33], [33, 83], [83, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34]]
[[89, 42], [42, 59], [59, 33], [33, 83], [83, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 32]]
[[89, 42], [42, 59], [59, 33], [33, 83], [83, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 73], [73, 18]]
[[89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 2], [2, 87], [87, 48], [48, 56], [56, 29], [29, 74], [74, 27]]
[[89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 2], [2, 87], [87, 48], [48, 56], [56, 29], [29, 74]]
[[89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 2], [2, 87], [87, 48], [48, 56], [56, 29]]
[[89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 2], [2, 87], [87, 48]

[[90, 94], [94, 93], [93, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53]]
[[90, 94], [94, 93], [93, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16]]
[[90, 94], [94, 93], [93, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1]]
[[90, 94], [94, 93], [93, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 71]]
[[90, 94], [94, 93], [93, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 71], [71, 86]]
[[90, 81], [81, 80], [80, 79], [79, 33], [33, 83], [83, 66], [66, 45], [45, 26], [26, 57], [57, 28], [28, 85], [85, 9], [9, 14], [14, 30]]
[[90, 81], [81, 80], [80, 79], [79, 33], [33, 83], [83, 66], [66, 45], [45, 26], [26, 57], [57, 28], [28, 85], [85, 9], [9, 14]]
[[90, 81], [81, 80], [80, 79], [79, 33], [33, 83], [83, 66], [66, 45], [45, 26], [26, 57], [

[[92, 20], [20, 91]]
[]
[[92, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 73], [73, 18], [18, '18b']]
[[92, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, '68b']]
[['18b', 27], [27, 74], [74, 29], [29, 56], [56, 48], [48, 87], [87, 2], [2, 53], [53, 7], [7, 17], [17, 4], [4, 40], [40, 69], [69, 38]]
[['18b', 27], [27, 74], [74, 29], [29, 56], [56, 48], [48, 87], [87, 2], [2, 53], [53, 7], [7, 17], [17, 4], [4, 40], [40, 69]]
[['18b', 27], [27, 74], [74, 29], [29, 56], [56, 48], [48, 87], [87, 2], [2, 53], [53, 7], [7, 17], [17, 4], [4, 40], [40, 69], [69, 38], [38, 70]]
[['18b', 27], [27, 74], [74, 29], [29, 56], [56, 48], [48, 87], [87, 2], [2, 53], [53, 7], [7, 60], [60, 54], [54, 35], [35, 25], [25, 5], [5, 36]]
[['18b', 27], [27, 74], [74, 29], [29, 56], [56, 48], [48, 87], [87, 2], [2, 53], [53, 7], [7, 60], [60, 54], [54, 35], [35, 25], [25, 5]]
[['18b', 27], [27, 74], [74, 29], [29, 56], [56

In [10]:
count

8100

In [11]:
len(complete_edges)

66026

In [12]:
complete_edges[6]

[36, 5]

In [13]:
complete = pd.DataFrame(complete_edges, columns=['o', 'd'])
complete['w'] = 1
complete_count = pd.DataFrame(complete.groupby(['o','d']).count())
complete_count.reset_index(inplace=True)
complete_count['o'] = complete_count['o'].astype(str)
complete_count['d'] = complete_count['d'].astype(str)

In [14]:
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='o', right_on='node_id')
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='d', right_on='node_id')
complete_count['geometry'] = [LineString([x,y]) for x, y in zip(complete_count['Point_x'], complete_count["Point_y"])]
complete_count = complete_count.loc[:,['o','d','w','geometry']]

In [15]:
complete_count

,o,d,w,geometry
0,1,10,830,"LINESTRING (51.19423615 43.6500856, 49.8663731..."
1,62,10,744,"LINESTRING (44.82710578 41.71513710000001, 49...."
2,67,10,382,"LINESTRING (52.96318620000001 40.02016264, 49...."
3,1,16,942,"LINESTRING (51.19423615 43.6500856, 55.1818477..."
4,53,16,946,"LINESTRING (60.31394877 47.04978092, 55.181847..."
...,...,...,...,...
203,94,90,63,"LINESTRING (71.56378993 34.10942691, 67.001097..."
204,89,93,207,"LINESTRING (69.19380137 34.50670823, 71.111711..."
205,94,93,134,"LINESTRING (71.56378993 34.10942691, 71.111711..."
206,90,94,63,"LINESTRING (67.00109753 24.89698664, 71.563789..."


In [16]:
complete_count.to_csv(complete_volume_map)

# Map specific routes with trade flows

In [18]:
#trade_flows = pd.read_csv(os.path.join(res_folder, "trade_flows_v1.csv"))
complete_edges = []
for idx, row in trade_flows.iterrows():    
    
    try:
        origin = row['Node number']
        #print(origin)
        
            
        destination = row['Node Number']
        route = nx.bidirectional_shortest_path(G, origin, destination)
        if origin == 91:
            print(origin,destination,route)
        edge_table = tabulate_edges(route)
        [x.append(int(row['Trade volume'].replace(",", ""))) for x in edge_table]
        complete_edges = complete_edges + edge_table 
    except:
        pass
        

    


91 89 [91, 20, 92, 81, 90, 94, 93, 89]
91 90 [91, 20, 92, 81, 90]
91 36 [91, 20, 76, 12, 66, 45, 55, 58, 43, '68b', 68, 39, 54, 35, 25, 5, 36]
91 4 [91, 20, 76, 12, 66, 45, 55, 58, 43, '68b', 68, 39, 54, 60, 7, 17, 4]
91 22 [91, 20, 76, 12, 66, 83, 33, 59, 42, 31, 65, 37, 22]
91 25 [91, 20, 76, 12, 66, 45, 55, 58, 43, '68b', 68, 39, 54, 35, 25]
91 60 [91, 20, 76, 12, 66, 45, 55, 58, 43, '68b', 68, 39, 54, 60]
91 54 [91, 20, 76, 12, 66, 45, 55, 58, 43, '68b', 68, 39, 54]
91 63 [91, 20, 76, 12, 66, 45, 63]
91 10 [91, 20, 76, 12, 66, 45, 55, 58, 43, 8, 67, 10]


In [42]:
len(complete_edges)

385

In [43]:
complete_edges

[[36, 5, 18214],
 [5, 25, 18214],
 [25, 35, 18214],
 [35, 54, 18214],
 [54, 37, 18214],
 [37, 65, 18214],
 [65, 31, 18214],
 [31, 42, 18214],
 [42, 89, 18214],
 [36, 5, 373],
 [5, 25, 373],
 [25, 35, 373],
 [35, 54, 373],
 [54, 37, 373],
 [37, 65, 373],
 [65, 31, 373],
 [31, 42, 373],
 [42, 89, 373],
 [89, 93, 373],
 [93, 94, 373],
 [94, 90, 373],
 [36, 70, 6957371],
 [70, 38, 6957371],
 [38, 69, 6957371],
 [69, 40, 6957371],
 [40, 4, 6957371],
 [36, 5, 1737602],
 [5, 25, 1737602],
 [25, 35, 1737602],
 [35, 54, 1737602],
 [54, 37, 1737602],
 [37, 22, 1737602],
 [36, 5, 1218590],
 [5, 25, 1218590],
 [36, 5, 1218590],
 [5, 25, 1218590],
 [25, 35, 1218590],
 [35, 54, 1218590],
 [54, 60, 1218590],
 [36, 5, 1218590],
 [5, 25, 1218590],
 [25, 35, 1218590],
 [35, 54, 1218590],
 [91, 20, 1078810],
 [20, 92, 1078810],
 [92, 81, 1078810],
 [81, 90, 1078810],
 [90, 94, 1078810],
 [94, 93, 1078810],
 [93, 89, 1078810],
 [91, 20, 687606],
 [20, 76, 687606],
 [76, 12, 687606],
 [12, 66, 687606],
 [6

In [44]:
complete = pd.DataFrame(complete_edges, columns=['o', 'd', 'w'])
complete_count = pd.DataFrame(complete.groupby(['o','d']).sum())
complete_count.reset_index(inplace=True)
complete_count['o'] = complete_count['o'].astype(str)
complete_count['d'] = complete_count['d'].astype(str)

In [45]:
complete_count

,o,d,w
0,1,10,9906
1,4,17,26557061
2,5,25,5800427
3,5,36,1108161
4,7,17,5474893
...,...,...,...
75,93,94,373
76,94,90,373
77,94,93,22260730
78,68b,43,1022230


In [46]:
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='o', right_on='node_id')
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='d', right_on='node_id')
complete_count['geometry'] = [LineString([x,y]) for x, y in zip(complete_count['Point_x'], complete_count["Point_y"])]
complete_count = complete_count.loc[:,['o','d','w','geometry']]

In [47]:
complete_count

,o,d,w,geometry
0,1,10,9906,"LINESTRING (51.19423615 43.6500856, 49.8663731..."
1,4,17,26557061,"LINESTRING (76.85773523 43.22966422, 73.916840..."
2,7,17,5474893,"LINESTRING (68.8130505 42.43365853, 73.9168401..."
3,5,25,5800427,"LINESTRING (72.28458996000001 40.8147501, 71.7..."
4,35,25,6163925,"LINESTRING (67.85315559 40.13305512, 71.797853..."
...,...,...,...,...
75,89,93,373,"LINESTRING (69.19380137 34.50670823, 71.111711..."
76,94,93,22260730,"LINESTRING (71.56378993 34.10942691, 71.111711..."
77,90,94,22260730,"LINESTRING (67.00109753 24.89698664, 71.563789..."
78,93,94,373,"LINESTRING (71.11171118999999 34.19162304, 71...."


In [48]:
complete_count.to_csv(trade_volume_map)